In [1]:
from datetime import datetime
from sqlalchemy import (Column, Integer, Numeric, String, Table,
                        ForeignKey, create_engine, Date) 
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base 
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:')
Base = declarative_base()
Session = sessionmaker(bind=engine)

In [2]:
usersfacilitydepartments_table = Table('usersfacilitydepartments', 
                                       Base.metadata,
                                       Column('user_id', Integer, ForeignKey('users.user_id'), primary_key=True),
                                       Column('fd_id', Integer, ForeignKey('facility_departments.fd_id'), primary_key=True)
                                      )
userscommittees_table = Table('userscommittees', 
                               Base.metadata,
                               Column('user_id', Integer, ForeignKey('users.user_id'), primary_key=True),
                               Column('committee_id', Integer, ForeignKey('committees.committee_id'), primary_key=True)
                              )

In [3]:
class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer, primary_key=True)
    name = Column(String(255), index=True)
    start_date = Column(Date)
    obsolete_date = Column(Date)
    fd = relationship('FacilityDepartment', 
                      secondary=usersfacilitydepartments_table,
#                       primaryjoin=user_id==usersfacilitydepartments_table.c.user_id,
#                       secondaryjoin='FacilityDepartment.fd_id'==usersfacilitydepartments_table.c.fd_id,
                      backref='users')
    coms = relationship('Committee', 
                        secondary=userscommittees_table,
#                         primaryjoin=user_id==userscommittees_table.c.user_id,
                        backref='users')

In [4]:
class FacilityDepartment(Base):
    __tablename__ = 'facility_departments'
    
    fd_id = Column(Integer, primary_key=True)
    facility_name = Column(String(255))
    department_name = Column(String(255))
    start_date = Column(Date)
    obsolete_date = Column(Date)
    
#     fd = relationship('User', 
#                       secondary=usersfacilitydepartments_table,
#                       primaryjoin=user_id==usersfacilitydepartments_table.c.user_id,
#                       secondaryjoin=fd_id==usersfacilitydepartments_table.c.fd_id)


In [5]:
class Committee(Base):
    __tablename__ = 'committees'
    
    committee_id = Column(Integer, primary_key=True)
    name = Column(String(255))
    start_date = Column(Date)
    obsolete_date = Column(Date)


In [6]:
Base.metadata.create_all(engine)
session = Session()

In [7]:
from datetime import date
import random
start_date = date.today().replace(day=1, month=1).toordinal()
end_date = date.today().toordinal()

facilities = {'LA': ['Bakery', 'Dough Factory'],
              'NYC': ['Store', 'Online'],
              'BOS': ['Bakery', 'Dough Factory'],
              'BNA': ['Store', 'Online'],
             }

for x in xrange(20):
    random_day = date.fromordinal(random.randint(start_date, end_date))
    random_choice = random.choice(facilities.keys())
    for department in facilities[random_choice]:
        fd = FacilityDepartment(facility_name=random_choice, 
                                department_name=department,
                                start_date=date.fromordinal(start_date),
                                obsolete_date=random_day)
        session.add(fd)
session.commit()

In [8]:
committees = ['Agriculture', 'Appropriations', 'Armed Services', 'Budget']
for committee in committees:
    random_day = date.fromordinal(random.randint(start_date, end_date))
    com = Committee(name=committee, 
                    start_date=date.fromordinal(start_date),
                    obsolete_date=random_day
                   )
    session.add(com)
session.commit()

In [9]:
names = ['George Washington', 'John Adams', 'Thomas Jefferson', 
         'James Madison', 'James Monroe', 'John Quincy Adams', 
         'Andrew Jackson', 'Martin Van Buren', 'William Henry Harrison',
         'John Tyler', 'James Knox Polk', 'Zachary Taylor']
for name in names:
    random_day = date.fromordinal(random.randint(start_date, end_date))
    user = User(name=name, 
                    start_date=date.fromordinal(start_date),
                    obsolete_date=random_day
                   )
    session.add(user)
session.commit()

In [10]:
user = session.query(User).first()

In [11]:
user.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10cd04590>,
 'name': u'George Washington',
 'obsolete_date': datetime.date(2016, 4, 12),
 'start_date': datetime.date(2016, 1, 1),
 'user_id': 1}

In [12]:
for user in session.query(User).all():
    for x in xrange(3):
        fd = random.choice(session.query(FacilityDepartment).all())
        user.fd.append(fd)
    session.add(user)
session.commit()

In [13]:
for user in session.query(User).all():
    for x in xrange(3):
        com = random.choice(session.query(Committee).all())
        user.coms.append(com)
    session.add(user)
session.commit()

In [26]:
random_day = date.fromordinal(random.randint(start_date, end_date))
print random_day
query = session.query(FacilityDepartment.users.name, 
                      FacilityDepartment.facility_name, 
                      FacilityDepartment.department_name).join(
#     usersfacilitydepartments_table,
    FacilityDepartment).filter(
    FacilityDepartment.obsolete_date<=random_day).group_by(User.name, FacilityDepartment.facility_name, FacilityDepartment.department_name)

print(query)
for result in query:
    print result

2016-01-26


AttributeError: Neither 'InstrumentedAttribute' object nor 'Comparator' object associated with FacilityDepartment.users has an attribute 'name'

In [21]:
for x in user.fd:
    print x.__dict__

{'obsolete_date': datetime.date(2016, 1, 14), '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10ce4c490>, 'facility_name': u'NYC', 'department_name': u'Online', 'fd_id': 12, 'start_date': datetime.date(2016, 1, 1)}
{'obsolete_date': datetime.date(2016, 3, 10), '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10cd848d0>, 'facility_name': u'BNA', 'department_name': u'Store', 'fd_id': 25, 'start_date': datetime.date(2016, 1, 1)}
{'obsolete_date': datetime.date(2016, 3, 22), '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10ce4c390>, 'facility_name': u'LA', 'department_name': u'Dough Factory', 'fd_id': 40, 'start_date': datetime.date(2016, 1, 1)}


In [22]:
user.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10cd2dc90>,
 'fd': [<__main__.FacilityDepartment at 0x10cd59d90>,
 'name': u'Zachary Taylor',
 'obsolete_date': datetime.date(2016, 4, 21),
 'start_date': datetime.date(2016, 1, 1),
 'user_id': 12}